<a href="https://colab.research.google.com/github/BMugo84/pytorch_in_25_hours/blob/main/attn_is_all_you_need_Shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# Download the dataset tiny Shakespear Dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-09-12 01:00:25--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2024-09-12 01:00:25 (37.4 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [26]:
# read it in to inspect it
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [27]:
print("Length of dataset in characters: ", len(text))

Length of dataset in characters:  1115394


In [28]:
# looking at the first 1000 chars
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [29]:
# Check all unique characters in the test
chars = sorted(list(set(text)))
print(''.join(chars))
print(len(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [30]:
# create a mapping for characters to integers
stoi = {ch:i for i, ch in enumerate(chars)} # string to integer
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # ie take a string , output a list of integers
decode = lambda l: "".join([itos[i] for i in l]) # ie take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [31]:
# # expanding the top code
# stoi = {}
# for i,ch in enumerate(chars):
#     stoi[ch] = i
# itos = {}
# for i, ch in enumerate(chars):
#     itos[i] = ch

# def encodetst(s):
#     intlist = []
#     for c in s:
#         intlist.append(stoi[c])
#     return intlist
# def decodetst(l):
#     strlist = []
#     for i in l:
#         strlist.append(itos[i])
#     return "".join(strlist)

# print(encodetst("hii there"))
# print(decodetst(encodetst("hii there")))

In [32]:
# testing tiktoken
!pip install tiktoken

import tiktoken
enc = tiktoken.get_encoding("o200k_base")
assert enc.decode(enc.encode("hello world")) == "hello world"

# To get the tokeniser corresponding to a specific model in the OpenAI API:
enc = tiktoken.encoding_for_model("gpt-4o")

In [33]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")
print(enc.encode("hii there"))

[71, 4178, 612]


In [34]:
# lets now encode the entire text dataset and store it into a torch.Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [35]:
# lets split the data into train and test datasets/validation
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [36]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [37]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [24]:
# introducing batch dimension
torch.manual_seed(1337)
batch_size = 4 # how many independent sequence will we process in parallel
block_size = 8 # what is the max context length for prediction

def get_batch(split):

